# RescueNet — Attention U-Net Training
**Teknofest 2026 | Sentinair Team**

Dataset: [yaroslavchyryo/rescuenet](https://www.kaggle.com/datasets/yaroslavchyrko/rescuenet)  
Model: Attention U-Net (98.47% mIoU on RescueNet paper)  
Input: 713×713 | Classes: 11

---
**Checkpoint yönetimi:**  
`/kaggle/working/checkpoints/` → Kaggle Output sekmesinde görünür, oturum kapanınca da erişilebilir kalır.  
Bir sonraki oturumda devam etmek için Output'u yeni oturuma dataset olarak ekle.

**İlk kez çalıştırıyorsan:** Hücreleri sırayla çalıştır (1→6).  
**Devam ediyorsan:** Sadece Hücre 1, 2, 4 (RESUME_PATH doldur) ve 6'yı çalıştır.

In [ ]:
# Hücre 1: Repo klonla
import os

REPO_URL = 'https://github.com/nickotyn46/RescueNetModel'  # <-- bunu değiştir
REPO_DIR = '/kaggle/working/RescueNetModel'

if not os.path.exists(REPO_DIR):
    os.system(f'git clone {REPO_URL} {REPO_DIR}')
else:
    print('Repo zaten mevcut, güncelleniyor...')
    os.system(f'cd {REPO_DIR} && git pull')

os.chdir(REPO_DIR)
print(f'Çalışma dizini: {os.getcwd()}')

In [ ]:
# Hücre 2: Bağımlılıkları kur
import os
os.system('pip install -q pyyaml tensorboard tqdm')
print('Kurulum tamamlandı.')

In [ ]:
# Hücre 3: GPU ve dataset kontrolü
import torch
print(f'PyTorch  : {torch.__version__}')
print(f'CUDA     : {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU      : {torch.cuda.get_device_name(0)}')
    print(f'VRAM     : {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

import os
DATA_ROOT = '/kaggle/input/datasets/yaroslavchyrko/rescuenet/RescueNet'
for split in ['train', 'val', 'test']:
    img_dir = os.path.join(DATA_ROOT, split)
    if os.path.exists(img_dir):
        n = sum(len(f) for _, _, f in os.walk(img_dir))
        print(f'{split:5s}: {n} dosya bulundu')
    else:
        print(f'UYARI: {img_dir} bulunamadı — rescuenet datasetini bu notebook a eklemelisin!')

In [ ]:
# Hücre 4: Config ayarla
# ─────────────────────────────────────────────────────────────────────
# DEVAM EDİYORSAN: RESUME_PATH'i doldur.
#   1. Önceki oturumun Output'undaki checkpoints klasörünü
#      yeni oturuma "Add Data" ile ekle
#   2. RESUME_PATH'i aşağıdaki gibi ayarla:
#      RESUME_PATH = '/kaggle/input/DATASET_ADI/checkpoints/latest.pth'
# ─────────────────────────────────────────────────────────────────────
RESUME_PATH = ''  # Boş bırak = sıfırdan başla

import yaml, os

config_path = 'configs/rescuenet_aunet.yaml'
with open(config_path, 'r') as f:
    cfg = yaml.safe_load(f)

cfg['DATA']['data_root']  = '/kaggle/input/datasets/yaroslavchyrko/rescuenet/RescueNet'
cfg['TRAIN']['save_path'] = '/kaggle/working/checkpoints'
cfg['TEST']['save_folder']= '/kaggle/working/predictions'
cfg['TRAIN']['resume']    = RESUME_PATH

os.makedirs('/kaggle/working/checkpoints', exist_ok=True)

with open(config_path, 'w') as f:
    yaml.dump(cfg, f, default_flow_style=False)

print('Config güncellendi.')
if RESUME_PATH:
    print(f'Devam noktası: {RESUME_PATH}')
    print(f'Dosya mevcut : {os.path.exists(RESUME_PATH)}')
else:
    print('Sıfırdan başlanacak.')

In [ ]:
# Hücre 5: Sanity check — bir batch yükle
import sys
sys.path.insert(0, '/kaggle/working/RescueNetModel')

from data.dataset import RescueNetDataset
from transforms import get_train_transform

ds = RescueNetDataset(
    root_dir='/kaggle/input/datasets/yaroslavchyrko/rescuenet/RescueNet',
    mode='train',
    joint_transform=get_train_transform(713, 713)
)
print(f'Train seti boyutu: {len(ds)}')
img, mask = ds[0]
print(f'Görüntü: {img.shape}  {img.dtype}')
print(f'Maske  : {mask.shape}  {mask.dtype}')
print(f'Sınıflar: {mask.unique().tolist()}')

In [ ]:
# Hücre 6: TensorBoard'u başlat (eğitimle paralel çalışır)
# Bu hücreyi çalıştırdıktan sonra 7. hücreyi çalıştır.
# Eğitim ilerledikçe grafik otomatik güncellenir.
%load_ext tensorboard
%tensorboard --logdir /kaggle/working/checkpoints/logs

In [ ]:
# Hücre 7: EĞİTİMİ BAŞLAT
# Checkpointler: /kaggle/working/checkpoints/
#   best.pth   → en iyi val mIoU
#   latest.pth → her 5 epoch'ta güncellenir (kaldığın yer)
#
# Oturum kapanmadan ÖNCE sağdaki Output sekmesinden
# checkpoints klasörünü indir veya "Save Version" yap!

import os
os.system('python train.py --config configs/rescuenet_aunet.yaml')

In [ ]:
# Hücre 8: Checkpoint durumunu kontrol et
import os, torch

ckpt_dir = '/kaggle/working/checkpoints'
for fname in ['best.pth', 'latest.pth']:
    path = os.path.join(ckpt_dir, fname)
    if os.path.exists(path):
        ckpt = torch.load(path, map_location='cpu')
        print(f'{fname:12s}  epoch={ckpt["epoch"]:3d}  best_mIoU={ckpt["best_miou"]:.4f}  boyut={os.path.getsize(path)/1e6:.1f}MB')
    else:
        print(f'{fname:12s}  bulunamadı')

In [ ]:
# Hücre 9: Test setinde değerlendir
import os
os.system(
    'python evaluate.py '
    '--config configs/rescuenet_aunet.yaml '
    '--model-path /kaggle/working/checkpoints/best.pth '
    '--save-masks'
)

In [ ]:
# Hücre 10: ONNX export (Hailo için)
import os
os.system(
    'python export_onnx.py '
    '--config configs/rescuenet_aunet.yaml '
    '--model-path /kaggle/working/checkpoints/best.pth '
    '--output /kaggle/working/rescuenet_aunet.onnx'
)

# Output klasöründeki dosyaları listele
print('\n── Output dosyaları ──')
for root, dirs, files in os.walk('/kaggle/working'):
    for f in sorted(files):
        path = os.path.join(root, f)
        rel  = path.replace('/kaggle/working/', '')
        print(f'  {rel:50s}  {os.path.getsize(path)/1e6:.1f} MB')